# netCDFファイルをasciiファイルに変換せずに計算する

## 作業準備

#### 作業領域を指定する

In [1]:
# GoogleDrive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 作業ディレクトリを移動する
%cd /content/drive/MyDrive/seminar2022/卒業研究/nakamura/GraduationResearch/code/

/content/drive/MyDrive/seminar2022/卒業研究/nakamura/GraduationResearch/code


#### モジュールをインポートする

In [3]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from netCDF4 import Dataset
import netCDF4 as nc

#### データ構造の把握

In [4]:
# ファイルを読み込む
file = './data/v6.0_CO2_excl_short-cycle_org_C_2015_AGS.0.1x0.1.nc'
fh = Dataset(file, mode='r',format="NETCDF4")

In [5]:
print(fh)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.0
    title: Annual Mean Emissions of CO2 - 
    institution: European Commission, Joint Research Centre
    source: http://edgar.jrc.ec.europa.eu/
    history: Created from original data (0.1x0.1 degrees) using IDL program (edgar_ascii_to_ncdf.pro) on Tue Aug 03 09:34:50 2021
    references: European Commission, Joint Research Centre (JRC)/Netherlands Environmental Assessment Agency (PBL). Emission Database for Global Atmospheric Research (EDGAR),http://edgar.jrc.ec.europe.eu
    copyright_notice: Reproduction of the data is authorized, except for commercial purposes, provided the source is fully acknowledged in the from"Source: European Commission, Joint Research Centre (JRC)/Netherlands Environmental Assessment Agency (PBL). Emission Databasefor Global Atmospheric Research (EDGAR), http://edgar.jrc.ec.europe.eu". Where reproduction is on a web-site,at least one link

In [6]:
# 格納されているパラメータを調べる
print(fh.variables.keys())

dict_keys(['lat', 'lon', 'emi_co2'])


## 計算

#### 値を緯度ごとに積算する

実行に35分かかった

In [7]:
# 値の緯度ごとの合計をリストに格納する
value_list = []
lon = 0; lat = 0    # lon = 経度 lat = 緯度
for lat in range(1800):
    list_lat = []
    for lon in range(3600):
        if fh['emi_co2'][lat][lon] >= 0:
            list_lat.append(float(fh['emi_co2'][lat][lon]))
        else:
            list_lat.append(float(0))
    value_list.append(sum(list_lat))
print(value_list)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

#### データフレームを作成する

In [8]:
# グリッドの格子間隔
grid_value = 1800 / 180

In [9]:
# 緯度を作成する
x = []; i = 0
for i in range(1800):
    x.append(-90 + i/grid_value)
print(x) 

[-90.0, -89.9, -89.8, -89.7, -89.6, -89.5, -89.4, -89.3, -89.2, -89.1, -89.0, -88.9, -88.8, -88.7, -88.6, -88.5, -88.4, -88.3, -88.2, -88.1, -88.0, -87.9, -87.8, -87.7, -87.6, -87.5, -87.4, -87.3, -87.2, -87.1, -87.0, -86.9, -86.8, -86.7, -86.6, -86.5, -86.4, -86.3, -86.2, -86.1, -86.0, -85.9, -85.8, -85.7, -85.6, -85.5, -85.4, -85.3, -85.2, -85.1, -85.0, -84.9, -84.8, -84.7, -84.6, -84.5, -84.4, -84.3, -84.2, -84.1, -84.0, -83.9, -83.8, -83.7, -83.6, -83.5, -83.4, -83.3, -83.2, -83.1, -83.0, -82.9, -82.8, -82.7, -82.6, -82.5, -82.4, -82.3, -82.2, -82.1, -82.0, -81.9, -81.8, -81.7, -81.6, -81.5, -81.4, -81.3, -81.2, -81.1, -81.0, -80.9, -80.8, -80.7, -80.6, -80.5, -80.4, -80.3, -80.2, -80.1, -80.0, -79.9, -79.8, -79.7, -79.6, -79.5, -79.4, -79.3, -79.2, -79.1, -79.0, -78.9, -78.8, -78.7, -78.6, -78.5, -78.4, -78.3, -78.2, -78.1, -78.0, -77.9, -77.8, -77.7, -77.6, -77.5, -77.4, -77.3, -77.2, -77.1, -77.0, -76.9, -76.8, -76.7, -76.6, -76.5, -76.4, -76.3, -76.2, -76.1, -76.0, -75.9, -75.8

In [10]:
# データフレームに書き出す
df = pd.DataFrame(columns=['lat', 'value'])  # 空のデータフレームを作る
df['lat'] = x   #データフレームに追加する
df['value'] = value_list

#### 緯度の幅を調整する

In [11]:
latitude = 2    # 何度間隔で積算したいか
lat_value = latitude * 10

In [12]:
# 積算する
sum_list = []
for i in range(len(df['value'])):
    fig_list = []; sum_value = 0
    if i % lat_value == 0:
        for n in range(lat_value):
            fig_list.append(df['value'][i + n])
        sum_value = sum(fig_list)
        sum_list.append(sum_value)
print(sum_list)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.863048090932072e-09, 1.4129631061386006e-08, 1.8889101268076717e-08, 1.990960357692584e-08, 4.8722722253216585e-08, 1.297651398557331e-07, 1.0724553639484003e-07, 9.561824987226374e-08, 6.286271880564109e-07, 4.0760528369523905e-07, 2.0343451835138023e-07, 2.2577996260149395e-07, 4.0100418558413943e-07, 6.963452614253618e-07, 1.1033164014906372e-06, 6.5161529307367e-07, 8.224856200509244e-07, 7.085210683773668e-07, 5.837203129846831e-07, 4.035170457124168e-07, 3.957534435426921e-07, 2.828614375365171e-07, 2.8508003989101724e-07, 3.7647316879520357e-07, 5.726770881069796e-07, 2.574767840453456e-07, 3.2586287281625296e-07, 2.2785930777063016e-07, 2.0444249156385525e-07, 2.6590976441579505e-07, 3.686015743206831e-07, 5.385797830402257e-07, 3.9692831737693975e-07, 5.603706213176797e-07, 4.245446071576062e-07, 7.493989195248885e-07, 1.0862823424618517e-06, 8.987959736876414e-07, 1.0116502103327402e-06, 1

In [13]:
# 緯度を作成する
n = []; i = 0
for i in range(90):
    n.append(-90 + latitude * i)
print(n) 

[-90, -88, -86, -84, -82, -80, -78, -76, -74, -72, -70, -68, -66, -64, -62, -60, -58, -56, -54, -52, -50, -48, -46, -44, -42, -40, -38, -36, -34, -32, -30, -28, -26, -24, -22, -20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88]


In [14]:
# データフレームに書き出す
df_lat = pd.DataFrame(columns=['lat', 'value'])  # 空のデータフレームを作る
df_lat['lat'] = n   #データフレームに追加する
df_lat['value'] = sum_list
print(df_lat)

    lat  value
0   -90    0.0
1   -88    0.0
2   -86    0.0
3   -84    0.0
4   -82    0.0
..  ...    ...
85   80    0.0
86   82    0.0
87   84    0.0
88   86    0.0
89   88    0.0

[90 rows x 2 columns]
